In [1]:
from app import Manga, MangaChapter, db

db.create_all()  

# Ajouter les manga à la table de base
one_piece = Manga(name = "One Piece",url = "https://www.scan-vf.net/one_piece/", cover="https://www.glenat.com/sites/default/files/images/livres/couv/9782344049020-001-T.jpeg")
boruto = Manga(name = "Boruto", url= "https://www.scan-vf.net/boruto", cover="https://images-na.ssl-images-amazon.com/images/I/81cDaskL55L.jpg")
fairytail100 = Manga(name = "Fairy Tail 100 Years Quest", url= "https://www.scan-vf.net/fairy-tail-100-years-quest", cover="https://pbs.twimg.com/media/FMbhQ8DXoAAFVgI?format=jpg&name=large")
onepunchman = Manga(name = "One Punch Man", url= "https://www.scan-vf.net/one-punch-man", cover="https://static.fnac-static.com/multimedia/Images/FR/NR/16/ec/bf/12577814/1507-1/tsp20210803073801/One-Punch-Man.jpg")
hxh = Manga(name = "Hunter X Hunter", url= "https://www.scan-vf.net/hunter-x-hunter", cover="https://images-na.ssl-images-amazon.com/images/I/81gU6RgiRvL.jpg")
dragon_ball_super = Manga(name = "Dragon Ball Super", url= "https://www.scan-vf.net/dragon-Ball-Super", cover="https://www.db-z.com/wp-content/uploads/2021/11/cover-tome-17-dragon-ball-super-hd.jpg")

#J'ajoute juste le dernier élément :) Ne pas ajouter tout le reste en mode liste
db.session.add_all([one_piece,boruto,fairytail100,onepunchman,hxh,dragon_ball_super]) 

db.session.commit()

c:\Users\cleme\Documents\GitHub\Manwhen\venv\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [22]:
#cover_add = Manga.query.all()[0]
#cover_add.cover = 'https://www.scan-vf.net/uploads/manga/dragon-Ball-Super/cover/cover_250x350.jpg'
#db.session.commit()

In [38]:
from app import Manga, MangaChapter, db

manga_test = Manga.query.get(1)
#manga_test.id

manga_test_2 = db.session.query(Manga.id).filter_by(name='Hunter  Hunter').scalar() is not None
print(manga_test_2)

False


In [2]:
from app import Manga, MangaChapter, db
import requests
from bs4 import BeautifulSoup

for manga in Manga.query.all():

  # J'itère sur tous les manga
  html_text = requests.get(manga.url).text
  soup = BeautifulSoup(html_text, 'html.parser')

  #Je vais chercher les informations sur la page du manga en question
  online_chapters = soup.find_all('h5', {'class' : 'chapter-title-rtl'} )

  #Je cherche toutes les info (name, url, title)
  for num_chap in range(len(online_chapters)):

    last_chap_url = online_chapters[num_chap].find('a')['href']
    last_chap_title = online_chapters[num_chap].find('em').getText()

    last_chap_number_process = online_chapters[num_chap].find('a').getText()
    last_chap_number_process = last_chap_number_process.split()
    last_chap_number = last_chap_number_process[len(last_chap_number_process)-1]

    state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()

    if state is None :
        var = MangaChapter(chapter_number = last_chap_number,
                   chapter_url = last_chap_url, 
                   chapter_name = last_chap_title, 
                   manga = manga)
        db.session.add(var)

db.session.commit()
    
